In [3]:
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import linregress
from scipy import stats

In [43]:
rent_df = pd.read_excel("Metro_zori_uc_sfr_sm_month.xlsx", sheet_name="Filtered_clean")
print(rent_df.head())

principal_value_df = pd.read_excel("Metro_zhvi_uc_sfr_tier_0.33_0.67_sm_sa_month.xlsx", sheet_name="Filtered_cleaned")
principal_value_df.head()

   MonthYear  Minneapolis, MN   Duluth, MN  Rochester, MN
0 2018-01-31      1651.229431  1285.898298    1324.459656
1 2018-02-28      1666.570082  1282.255189    1373.733893
2 2018-03-31      1675.741524  1273.994196    1394.786511
3 2018-04-30      1688.432775  1272.153748    1420.143102
4 2018-05-31      1695.923052  1288.003275    1416.106843


,MonthYear,"Minneapolis, MN","Duluth, MN","Rochester, MN"
0,2018-01-31,272790.391700,153019.204829,218363.967261
1,2018-02-28,274889.696695,153307.688686,219632.614341
2,2018-03-31,277166.902005,153919.747485,220774.265507
3,2018-04-30,279441.449079,154623.054771,221707.680088
4,2018-05-31,281434.591075,155506.518195,222588.557622


In [44]:
# Rent

rent_df.head().round()

,MonthYear,"Minneapolis, MN","Duluth, MN","Rochester, MN"
0,2018-01-31,1651.0,1286.0,1324.0
1,2018-02-28,1667.0,1282.0,1374.0
2,2018-03-31,1676.0,1274.0,1395.0
3,2018-04-30,1688.0,1272.0,1420.0
4,2018-05-31,1696.0,1288.0,1416.0


In [45]:
# principal value

principal_value_df.head().round()

,MonthYear,"Minneapolis, MN","Duluth, MN","Rochester, MN"
0,2018-01-31,272790.0,153019.0,218364.0
1,2018-02-28,274890.0,153308.0,219633.0
2,2018-03-31,277167.0,153920.0,220774.0
3,2018-04-30,279441.0,154623.0,221708.0
4,2018-05-31,281435.0,155507.0,222589.0


In [53]:
# Assuming that the data extracrted is the home value and 30 yr mortgage, principal value
monthly_principal_payment_minneapolis = zhvi_df["Minneapolis, MN"]/30/12
monthly_principal_payment_minneapolis.head().round()

0    758.0
1    764.0
2    770.0
3    776.0
4    782.0
Name: Minneapolis, MN, dtype: float64